# 極性判定とDoc２Vecを使ったTwitterネガポジ予測
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
### 【このnotebookについて】
2019年7〜10月までフルタイムで通っていたスクールの卒業課題テーマを、機械学習の勉強のために発展させたものです<br>
卒業発表スライド　https://www.slideshare.net/secret/y0m7g1nZdxpVYP<br>
＊当初は炎上予測がテーマだったので、このnotebookの内容とはややズレます<br>
＊表紙スライドの字が見えない場合は２枚目から戻ると見えます<br>

＊ちなみに…<br>
スクールで取り組んだ課題のリポジトリ 
https://github.com/kaorisugi/diveintocode-ml<br>
論文読解課題のスライドシェア 
https://www.slideshare.net/secret/qGmdiwl4uGS20O<br>

### 【ゴール】
これからツイートする予定の文章に対し、過去の類似ツイートを探し、反応のネガポジスコア付きで上位１０位まで提示する。<br>
### 【モデルの仕組み】
１）ツイートデータセットを取得<br>
　・TwitterAPIを使ってツイートを取得<br>
　・各ツイートに対する反応ツイート（リプライ、引用RT）を取得<br>
　・反応ツイートの極性表現数をカウントしてネガポジスコアとpositive/negative/fire!!!判定を得る<br>
 　（positive/negativeの判定基準：極性表現数が多い方、fire!!!(炎上）の判定基準：極性表現の７０％以上がnegative）
２）データセットの前処理<br>
　・正規表現、ストップワード除去など<br>
３）予測モデルを生成<br>
　・データセットをDoc２vecで学習<br>
４）ツイート予定文章のネガポジ予測を返す<br>
　・データセットから、ツイート予定文書と似ている文書を探す<br>
 ・ネガポジスコア付きで、類似ツイート上位１０個を返す
### 【結果】
類似度確認用にデータセット内にあるものと同じ文を入力したところ、類似度1位で返ってきた。また、２位、３位にもマスクに関する似た話題のツイートが提示されたので類似ツイートの抽出は成功。ネガポジスコアもデータズレなどなく正確に表示され、目的は達成できた。<br>
ツイッターAPI制限により、まだサンプルが少ない（完成時２００件程度）が、データを蓄積できる仕様にしているので、ツイート文のバリエーションを増やしていけば、様々な入力文に対応できるようになると思う。<br>
ネガポジ判定については、ネガティブなテーマへの言及に共感したコメントでネガ判定が出ているケースも多く、必ずしもツイート主へのネガ感情ではないことに注意が必要。<br>

### 【その他試みたこと】
１）文章ベクトルを特徴量としたネガポジ予測モデル<br>
　・文章ベクトルとフォロワー数を特徴量X、ネガポジスコアを目的変数yとしたデータを学習<br>
　・文章ベクトルはDoc２vecとTf-idfの２種を作成<br>
　・ツイート予定文書を入力してネガポジスコアを予測する<br>
　・試した予測モデル<br>
　・MultiOutputRegressor、SVRのrbf と　SVRの線形、lightgbm、ランダムフォレスト<br>
　  　→精度が低すぎて断念<br>
２）ツイッターAPI制限への挑戦（データセットの拡大）<br>
　・古いツイートを大量取得できるパッケージを発見（通常は１週間程度しか遡れない）<br>
　　　→取得データから反応ツイートの取得を試みたができなかった<br>
   
### 【利用するには】
・config.py ファイルにツイッターAPIトークンを記入<br>
・インストールが必要なツールは、notebook内にマジックコマンドにて記載してあります<br>
・jupyternotebook上でのMeCabの利用で詰まる場合は、下記のDockerイメージを使うとうまくいくかと思います。<br>
Docker Japanese NLP<br>
https://github.com/hoto17296/docker-japanese-nlp<br>
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝

## １）ツイートデータセットを取得
・TwitterAPIでツイートを取得<br>
・各ツイートに対するリプライ、引用RTを取得<br>
・極性表現数をカウントしてネガポジスコアを得る<br>

#### 参考サイト
【Python】tweepyでTwitterのツイートを検索して取得<br>
https://vatchlog.com/tweepy-search/<br>
【Python】tweepyで期間指定してツイートを検索する<br>
https://vatchlog.com/tweepy-search-time/<br>
バズったツイートへのリアクションを感情分析してみる<br>【Google Natural Language API / Python】<br>
https://qiita.com/matsuri0828/items/029b4d0d510dcfb5c5dd

In [1]:
#必要なツールをインストール（初回のみ実行）
! pip install --upgrade pip
! pip install tweepy
! pip install oseti
! pip install requests requests_oauthlib
! pip install sengiri

Requirement already up-to-date: pip in /opt/conda/lib/python3.6/site-packages (19.3.1)


In [2]:
import tweepy
import re
import emoji
import oseti
from datetime import datetime, date, timedelta
import os
import pandas as pd
import csv
from tqdm import tqdm
import config

class Get_Twitter():

    def __init__(self, day, reload, print_rep = False, exclud_words = "配信スタート ＃キャンペーン　リツイートキャンペーン", RT_count = 5000):
        self.oseti_analyzer = oseti.Analyzer()  #極性判定
        self.CK = config.CONSUMER_KEY
        self.CS = config.CONSUMER_SECRET
        self.AT = config.ACCESS_TOKEN
        self.AS = config.ACCESS_TOKEN_SECRET
        self.ew = exclud_words
        self.print_rep = print_rep
        self.rt = str(RT_count)
        self.columns = [
            "Id", "Date", "Name", "Full_text",
            "Judge", "Posi_score", "Nega_score", "Followers", "link"
        ]
        self.posi_pd = pd.DataFrame([], columns = self.columns)
        self.nega_pd = pd.DataFrame([], columns = self.columns)
        self.fire_pd = pd.DataFrame([], columns = self.columns)
        self.wait = 0
        self.reload = reload
        day = datetime.strptime(day, '%Y-%m-%d')
        self.day = day.strftime('%Y-%m-%d')

    def main(self):
        self._Make_Dir() # データ格納ファイルの準備

        #ツイートを取得、センチメント判定
        try:
            status = self.Get_Buzz() #バズったツイート取得
            for i in status:            
                if self.wait == 10:
                    print("10回待機したため終了")
                    break
                self.Status(i)
                if self.Exclude_Word(self.buzz_full_text) == True:# 除外ワードを含むツイートは除外
                    continue
                if self.Text_Count() == True: #30W以下のツイートは除外
                    continue
                self.Get_Rep() #リプライを取得
                self.Get_RT() #RTコメントを取得
                if self.Min_Rep() == False: # コメントが少ないツイートは除外
                    continue
                self.Get_Senti() #コメントをセンチメント判定
                self._Get_Analysis() #ツイートをセンチメント判定
        #エラー時はスキップして次のツイート取得
        except (ValueError,  KeyError, TypeError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            if self.reload:
                self.wait += 1
                print("==========")
                print('get_buzzのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass
        
        #生成したデータをprint
        print()
        print("↓↓↓positiveサンプル↓↓↓")
        display(self.posi_pd.head())
        print()            
        print("↓↓↓negativeサンプル↓↓↓")
        display(self.nega_pd.head())
        print()
        print("↓↓↓fire_tweetサンプル↓↓↓")
        display(self.fire_pd.head())
        print()
        print()
        
        #生成したPandasDataFrameをcsvで書き出す
        total_pd = pd.concat([self.posi_pd, self.nega_pd, self.fire_pd], ignore_index=True)
        buzz_old = pd.read_csv('./output/buzz_tweet.csv')
        buzz_new = pd.concat([buzz_old, total_pd])#既存データと連結
        buzz_new.drop_duplicates(subset="Id",inplace=True)#重複ID行を削除            
        buzz_new.to_csv('./output/buzz_tweet.csv', index = False, header = True)
        print("csvへの書き出しが完了しました。新規データ数{}、全データ数：{}".format(len(buzz_new) - len(buzz_old), len(buzz_new)))
        print("サンプルが0件の場合は、15分後に再度実行すると取得できる場合があります。") 
        print("fire_tweetは出現率が非常に低いです。")

    #Api認証
    def _Auth(self):
        auth = tweepy.OAuthHandler(self.CK, self.CS)
        auth.set_access_token(self.AT, self.AS)
        api = tweepy.API(auth)
        return api

    #出力用ディレクトリとcsvファイルを作成（存在しない場合のみ）
    def _Make_Dir(self):
        new_dir_path = 'output'
        try:
            os.makedirs(new_dir_path)
        except FileExistsError:
            pass
        if (os.path.isfile('./output/buzz_tweet.csv')) == False:
            self.posi_pd.to_csv('./output/buzz_tweet.csv', index = False)  

    #絵文字削除
    def _remove_emoji(self, text):
        return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)

    #テキストを正規表現処理、絵文字削除
    def _format_text(self, text):
        text=re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
        text=re.sub('\n', "", text)
        text=re.sub(r'@?[!-~]+', "", text)
        text=self._remove_emoji(text)
        return text
    
    #　日付表記を整える、日本時間に修正
    def _date_format(self, date):
        date = datetime.strptime(str(date), '%a %b %d %H:%M:%S %z %Y')
        date = date + timedelta(hours=9)
        return datetime.strftime(date, '%Y-%m-%d %H:%M')

    def Status(self, status): 
        self.buzz_id = status._json['id']
        self.buzz_id_str = status._json['id_str']
        self.buzz_name = status._json['user']['screen_name']
        self.buzz_full_text = status._json['full_text']
        self.date = status._json['created_at']
        self.date = self._date_format(self.date)
        self.favo = status._json['favorite_count']
        self.rt_count = status._json['retweet_count']
        api = self._Auth()
        self.followers = status._json['user']['followers_count']
        #self.followers = len(api.followers(status._json['user']['screen_name']))
    
    #除外ワード
    def Exclude_Word(self, text):                        
        if self.ew in str(text):
            print("==========")
            print("除外ワード")
            print()
            return True
        else:
            return False

    #ツイート内にリンクがあれば分割
    def Text_Count(self):
        if re.search("(https://t.co/\w+)", self.buzz_full_text) == None:
            self.link = None
        else:                   
            self.buzz_full_text = re.split("(https://t.co/\w+)", self.buzz_full_text)
            self.link = self.buzz_full_text[1]
            self.buzz_full_text = self.buzz_full_text[0]
        if len(self.buzz_full_text) < 30:
            return True

    #リプライ＋引用RTコメントが100未満のツイートは除外
    def Min_Rep(self):
        reply_texts_rows = []
        if self.rep_cnt + self.RTcomme_cnt > 100:
            reply_texts_rows.append(self.rep_row)
            reply_texts_rows.append(self.rt_row)
            return True
        else:
            return False

    #sentiment_listを一次元にし、ツイートごとの極性表現の総和の辞書にする
    def Get_Senti(self):
        self.sentiment_list = sum(self.sentiment_list, [])#１次元にする
        self.sentiment = dict((key, sum(d[key] for d in self.sentiment_list)) for key in self.sentiment_list[0])

    #バズったツイートを取得(デフォルト：5000RT以上)
    def Get_Buzz(self):
        api = self._Auth()
        try:       
            status = api.search(q = 'filter:safe min_retweets:' + self.rt + ' exclude:retweets until:' + self.day,
                lang ='ja', count =100, tweet_mode = 'extended', result_type = 'recent')
            return status
        #エラー時はスキップして次のツイート取得
        except (ValueError,  KeyError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except (tweepy.RateLimitError, tweepy.TweepError) as e:
            if self.reload:
                self.wait += 1
                print("==========")
                print('get_buzzのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass
        #return status
    
    #リプライを取得
    def Get_Rep(self):
        api = self._Auth()     
        query_reply = '@' + self.buzz_name + ' exclude:retweets'
        self.rep_row = []
        self.sentiment_list = []
        self.rep_cnt =0
        wait_cnt = 0
        try:
            for status_reply in api.search(q=query_reply, lang='ja', count=100):
                if status_reply._json['in_reply_to_status_id_str'] == self.buzz_id_str:
                    row = self._format_text(status_reply._json['text'])
                    #極性判定
                    sentiment_score = self.oseti_analyzer.count_polarity(str(row))#strにする
                    self.sentiment_list.append(sentiment_score)
                    self.rep_row.append(row)
                    self.rep_cnt += 1
                else:
                    pass
        #エラーはスキップして次のツイート取得
        except (ValueError,  KeyError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            self.wait += 1
            if self.reload:
                print("==========")
                print('get_repのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass

    # 引用RTを取得
    def Get_RT(self):
        api = self._Auth()
        query_quote = self.buzz_id_str + ' exclude:retweets'
        self.RTcomme_cnt = 0
        self.rt_row = []
        try:
            for status_quote in api.search(q=query_quote, lang='ja', count=100):
                if status_quote._json['id_str'] == self.buzz_id_str:
                    continue
                else:
                    row = self._format_text(status_quote._json['text'])
                #極性判定
                sentiment_score = self.oseti_analyzer.count_polarity(str(row))#strにする
                self.sentiment_list.append(sentiment_score)
                self.rt_row.append(row)
                self.RTcomme_cnt += 1
        #エラーはスキップして次のツイート取得
        except (ValueError,  KeyError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            self.wait += 1
            if self.reload:
                print("==========")
                print('get_rtのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass        

    #取得したTweetをprint
    def _Print(self):
        print("name：", self.buzz_name, "／フォロワー数：", self.followers)
        print("date：", self.date, "／ツイートID：", self.buzz_id_str)
        print("RT数：", self.rt_count, "／favorite数：", self.favo)
        print("リプライ数：", self.rep_cnt, "／RTコメント数(上限１００）：", self.RTcomme_cnt)
        if self.print_rep == True:
            print("リプライ\n", self.rep_row)
            print("RTコメント\n", self.rt_row)
        else:
            pass

    #センチメント判定結果を取得
    def _Get_Analysis(self):
        total = self.sentiment["positive"] + self.sentiment["negative"]
        if self.sentiment["positive"] >= self.sentiment["negative"]:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:positive】　　極性表現数", self.sentiment)
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "positive", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.posi_pd = self.posi_pd.append(s, ignore_index=True)
        elif self.sentiment["negative"]/total >= 0.7:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:fire!!!】　　極性表現数", self.sentiment)
            print("ネガ表現の割合{:.3g}".format(self.sentiment["negative"]/total))
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "fire", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.fire_pd = self.fire_pd.append(s, ignore_index=True)
        else:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:negative】　　極性表現数", self.sentiment)
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "negative", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.nega_pd = self.nega_pd.append(s, ignore_index=True)
        print()



### ツイートデータセット取得　実行

In [3]:
# 指定日のツイートを取得（API制限のため取得できるのは約一週間前のものまで）
day = '2019-12-28'
# リクエスト制限対応：True:リクエスト上限に達したら15分待機ののちツイート取得続行/ False:待機せずcsv取得
reload = True

#除外ワード
exclud_words = "配信スタート ＃キャンペーン　リツイートキャンペーン WWWWWWWWW"

#その他設定可能パラメータ
#リプライをprint（print_rep = True/Fals), 最低RT数(RT_count = 5000)

GT = Get_Twitter(day, reload, exclud_words)
GT.main()

【鬼滅の刃コラボ中！】
ローソン国際展示場駅前店では1日限定で「鬼滅の刃」コラボを実施中です！

キャラクタースタンディやポスターの展示、また入店音が炭治郎・禰豆子・善逸・伊之助のボイス（ランダム）となっておりますのでぜひチェックしてください！

#鬼滅の刃 

【判定:positive】　　極性表現数 {'positive': 19, 'negative': 19}
name： kimetsu_off ／フォロワー数： 962989
date： 2019-12-28 08:56 ／ツイートID： 1210710961771859968
RT数： 7096 ／favorite数： 40235
リプライ数： 1 ／RTコメント数(上限１００）： 100

どなたかが呟かれてましたけど、成人のADHDでは「何か思いつくと、それをタスクの一番最後に追加するのではなく、現在行っているタスクの一番上に常に置いてしまい、それに注意を占有されてしまうことから、その結果スケジュールが崩壊して死ぬ」という状態になる人が多い気がしますな。

【判定:positive】　　極性表現数 {'positive': 61, 'negative': 59}
name： noooooooorth ／フォロワー数： 15678
date： 2019-12-28 08:45 ／ツイートID： 1210708419159609344
RT数： 6443 ／favorite数： 23513
リプライ数： 24 ／RTコメント数(上限１００）： 100

若月健矢選手、立花理香さん

ご結婚おめでとうございます！
末長くお幸せに✨

バファローズ㊗️ポンタ

【判定:positive】　　極性表現数 {'positive': 71, 'negative': 3}
name： bs_ponta ／フォロワー数： 288623
date： 2019-12-28 08:34 ／ツイートID： 1210705650491113472
RT数： 7019 ／favorite数： 13577
リプライ数： 59 ／RTコメント数(上限１００）： 51

【ご報告】この度、みなさまにご報告したいことができました。ぜひご覧いただけますと幸いです。 

【判定:positive】　　極性表現数 {'positive': 14

Id              Date            Name  \
0  1210710961771859968  2019-12-28 08:56     kimetsu_off   
1  1210708419159609344  2019-12-28 08:45    noooooooorth   
2  1210705650491113472  2019-12-28 08:34        bs_ponta   
3  1210702646509588480  2019-12-28 08:22  RiccaTachibana   
4  1210696997839110144  2019-12-28 08:00     kimetsu_off   

                                           Full_text     Judge Posi_score  \
0  【鬼滅の刃コラボ中！】\nローソン国際展示場駅前店では1日限定で「鬼滅の刃」コラボを実施中で...  positive         19   
1  どなたかが呟かれてましたけど、成人のADHDでは「何か思いつくと、それをタスクの一番最後に追...  positive         61   
2  若月健矢選手、立花理香さん\n\nご結婚おめでとうございます！\n末長くお幸せに✨\n\nバ...  positive         71   
3     【ご報告】この度、みなさまにご報告したいことができました。ぜひご覧いただけますと幸いです。   positive        149   
4  【#本日12月28日は竈門禰豆子の誕生日!!】\n本日は、鬼でありながら鬼殺隊に所属する\n...  positive         76   

  Nega_score Followers                     link  
0         19    962989  https://t.co/TgagYYCcAU  
1         59     15678                     None  
2          3    288623                     None  
3         23    175428  https://t.co/hR8m5IHoBD  
4         12    962989  https://t.co/gOF5Qp6woi


↓↓↓negativeサンプル↓↓↓


Id              Date             Name  \
0  1210688744660975616  2019-12-28 07:27  comiketofficial   

                                           Full_text     Judge Posi_score  \
0  【注意喚起！】寝ている人から財布等を抜き取る犯罪が発生しているという連絡が警察からありました...  negative         62   

  Nega_score Followers  link  
0        111    221676  None


↓↓↓fire_tweetサンプル↓↓↓


Empty DataFrame
Columns: [Id, Date, Name, Full_text, Judge, Posi_score, Nega_score, Followers, link]
Index: []



csvへの書き出しが完了しました。新規データ数1、全データ数：314
サンプルが0件の場合は、15分後に再度実行すると取得できる場合があります。
fire_tweetは出現率が非常に低いです。


## ２）データセットの前処理
　・正規表現、ストップワード除去など
 
 #### 参考サイト
Pythonで全角・半角記号をまとめて消し去る<br>　http://prpr.hatenablog.jp/entry/2016/11/23/Python%E3%81%A7%E5%85%A8%E8%A7%92%E3%83%BB%E5%8D%8A%E8%A7%92%E8%A8%98%E5%8F%B7%E3%82%92%E3%81%BE%E3%81%A8%E3%82%81%E3%81%A6%E6%B6%88%E3%81%97%E5%8E%BB%E3%82%8B

In [4]:
#必要なツールをインストール(初回のみ実行)
! pip install gensim
! pip install natto-py
! pip install emoji

In [20]:
#ツイートデータを学習用に整形
#from natto import MeCab
import MeCab
import re
import pandas as pd
import pprint
import emoji
import neologdn
import urllib.request
import unicodedata
import string

class For_Model():
    
    def __init__(self, data, columns, out_file, mode, text, similar = None):
        self.mecab = MeCab.Tagger("-Owakati")
        self.data = data
        self.columns = columns
        self.out_file = out_file
        self.mode = mode
        self.text = text
        self.similar = str(similar)

    #データを読み込む
    def Load_tweets(self):        
        df = pd.read_csv(self.data, usecols = self.columns)
        print("読み込んだツイート", df.shape)
        
        #３０w以下のtweet行を削除
        index = []
        for i in range(len(df)):
            line = df.iloc[i]
            text = str(line["Full_text"])
            text = re.sub('https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
            text = re.sub('http?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
            line["Full_text"] = text
            if len(text) < 30:
                index.append(i)
        df_tweet = df.drop(df.index[index])
        df_tweet = df_tweet.reset_index(drop=True)
        print("30w以下削除後のツイート数", df_tweet.shape)
        display(df_tweet.head())
        
        #判定用テキストをリストの最後に追加
        tweets = []
        for i in df_tweet[self.text]:
            tweets.append(i)
        if self.similar == None:
            pass
        else:
            tweets.append(self.similar)
        return df_tweet, tweets

    def Stop_Words(self):
        # ストップワードをダウンロード
        url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
        urllib.request.urlretrieve(url, './output/stop_word.txt')

        with open('./output/stop_word.txt', 'r', encoding='utf-8') as file:
            stopwords = [word.replace('\n', '') for word in file.readlines()]

        #追加ストップワードを設定（助詞や意味のない平仮名１文字）
        add_words = ['あ','い','う','え','お','か','き','く','け','こ','さ','し','す','せ','そ','た','ち','つ','て','と',
                     'な','に','ぬ','ね','の','は','ひ','ふ','へ','ほ','ま','み','む','め','も','や','ゆ','よ',
                     'ら','り','る','れ','ろ','わ','を','ん','が','ぎ','ぐ','げ','ご','ざ','じ','ず','ぜ','ぞ',
                     'だ','ぢ','づ','で','ど','ば','び','ぶ','べ','ぼ','ぱ','ぴ','ぷ','ぺ','ぽ',
                     'くん','です','ます','ました','そして','でも','だから','だが','くらい','その','それ','かも',
                     'あれ','あの','あっ','そんな','この','これ','とか','とも','する','という','ござい',
                     'ので','なんて','たら', 'られ','たい','さて','てる','ください','なる','けど','でし',
                     'じゃん','だっ','なっ','でしょ', 'ある','って','こんな','ねえ'
                    ]
        stopwords = stopwords + add_words
        return stopwords

    def Tokenizer(self, text, stopwords):

        words = []
        text = self.mecab.parse(text)
        text = text.split(' ')
        for j in range(len(text)):
            if text[j] not in stopwords:
                words.append(text[j])
        return words

    def remove_emoji(self, text):
        return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)

    #記号削除
    def format_text(self, text):
        text = unicodedata.normalize("NFKC", text)  # 全角記号を半角へ置換
        # 記号を消し去るための魔法のテーブル作成
        table = str.maketrans("", "", string.punctuation  + "「」、。・*`+-|?#!()\[]<>=~/")
        text = text.translate(table)
        return text

    def main(self):
        tweets_num = 0
        stopwords = self.Stop_Words()
        df_tweet, tweets = self.Load_tweets()
        #ツイートを分かち書きしてcsvに出力(モード'a'はデータ追加、モード'w'は新規作成)
        with open('./output/' + self.out_file, self.mode) as f:
            for i in tweets:
                tweets_num += 1
                i = neologdn.normalize(i)
                i = re.sub('\n', "", i)
                i = re.sub(r'[!-~]', "", i)#半角記号,数字,英字
                i = re.sub(r'[︰-＠]', "", i)#全角記号
                i = self.format_text(i)#記号削除
                i = re.sub(r'[【】●ㅅ●Ф☆✩︎♡→←▼①②③④⑤『』ω《》∠∇∩♪∀◞ཀCщ≧≦ ́◤◢■◆★※↑↓〇◯○◎⇒▽◉Θ♫♬〃“”◇ᄉ⊂⊃д°]', "", i)
                #i = re.sub(r'[!-~、。‥…？！〜「」｢｣:：“”【】※♪♩♫♬『』→↓↑《》〈〉[]≧∇≦・゜・●ㅅ●´Д´°ω°•ω•★＊☆♡（）✔Θ∀´∀｀˘ω˘‼бωб￣▽￣◉→←▼①②③④⑤]', "", i)
                i = self.remove_emoji(i)
                i = self.Tokenizer(i, stopwords)
                i = ' '.join(i) #リストを文字列に変換
                i = str(i)
                f.write(i)

        print('CSV出力完了：'+ self.out_file)
        with open('./output/' + self.out_file) as f:
             wakati = f.read()

        print("学習用データに書き込んだツイート数（判定用ツイート含む）：", tweets_num)
        print()
        print("分かち書きサンプル\n", wakati[:500])
        return df_tweet


### 前処理の実行

In [6]:
#パラメータの設定

#取得したデータのパス
data = './output/buzz_tweet.csv'
#取得したい列名
columns = ["Followers", "Full_text","Posi_score", "Nega_score","Judge"]
#出力ファイル名
out_file = "train_buzz.txt"
#学習データの保存モード　'a'：追加／'w'：上書き
mode = 'w'
#ツイートテキストの列を指定
text = "Full_text"
#判定させたいツイート予定文書（類似度確認のため、データセット内にあるツイート文を使用）
similar = "イオンマスク禁止従業員の人嫌がるのわかるわ。\
インフルで出校停止中なんだけど薬効いて体元気だからイオン遊ばせに来た。みたいな事凄く多いんだよ。\
『店員が媒介にならないよう全店でマスク奨励してます。ご理解下さい』\
ってアナウンスされる方が余程良いのでイオンさん、マスク禁止撤回して"

FM = For_Model(data, columns, out_file, mode, text, similar)
df_tweet = FM.main()

読み込んだツイート (314, 5)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


30w以下削除後のツイート数 (259, 5)


Followers                                          Full_text     Judge  \
0     1564.0  絶対断らないと評判の病児保育室、助成金下りず2億円の赤字を出し閉鎖\n\n全国に2886カ所...  positive   
1     2217.0  全国の皆さんへ\nどうか皆様のお力を貸してください。\n\n１日も早く娘を助けたいです。\n...  positive   
2     3756.0  o0( 歴史上、さんざん他国の料理を魔改造してきた我が国が「寿司ポリス」などどは片腹痛い！あ...  positive   
3    10999.0  これすんごい。GABAのフォースリープ、ツイッターで人気だったから半信半疑で夜に3粒食べてみ...  positive   
4    56685.0  TVアニメ「Ｄｒ．ＳＴＯＮＥ」第2期制作が決定いたしました！第1期をご視聴・応援くださってい...  positive   

   Nega_score  Posi_score  
0       148.0       168.0  
1        50.0       129.0  
2        85.0       137.0  
3        78.0        97.0  
4        20.0       156.0

CSV出力完了：train_buzz.txt
学習用データに書き込んだツイート数： 260

分かち書きサンプル
 絶対 断ら ない 評判 病児保育 助成金 下り 赤字 出し 閉鎖 全国 病児保育 赤字 運営 おり 東海 キッズケア 助成金 求め 署名 集め 助成金 下りる あり ませ 社会保障 税金 使わ ませ 
全国 皆さん どうか 皆様 お力 貸し 早く 娘 助け 家族 揃っ 笑顔 クリスマス 新年 迎え 娘 目撃 情報 娘 繋がる 些細なこと 連絡 連絡先 大月 警察 暑 電話 もしくは 最寄り 警察 署 拡散希望 小倉 美咲 
歴史 さんざん 他国 料理 魔 改造 我が国 寿司 ポリス どど 片腹痛い あらゆる 文化 文化 独自 解釈 いい イノベーション 生む 思っ しかし ヘルシンキ 〈 クリスマス トッピング • バナナ 巻き寿司 〉 寛容 試さ いる 
すん ごい フォースリープツイッター 人気 半信半疑 夜 粒 食べ 昨日 めちゃ ぐっすり 眠れ 例える なら 旅行 気持ち良く 遊び 疲れ 夜 睡眠 クオリティ しかも 味 まろやか ミルク チョコ 好み すぎる 目安 一日 粒 摂 れる 
アニメ 期 制作 決定 いたし 期 視聴 応援 くださっ いる 皆様 本当に ありがとう 


## ３）予測モデルを生成
　・データセットをDoc２vecで学習<br>

#### 参考サイト
fastTextとDoc2Vecのモデルを作成してニュース記事の多クラス分類の精度を比較する<br> https://qiita.com/kazuki_hayakawa/items/ca5d4735b9514895e197<br>

In [7]:
#Doc2Vecモデルの学習

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

f = open('./output/train_buzz.txt','r')#空白で単語を区切り、改行で文書を区切っているテキストデータ

#１文書ずつ、単語に分割してリストに入れていく[([単語1,単語2,単語3],文書id),...]こんなイメージ
#words：文書に含まれる単語のリスト（単語の重複あり）
# tags：文書の識別子（リストで指定．1つの文書に複数のタグを付与できる）
#fにテキスト データをいれる
trainings = [TaggedDocument(words = data.split(),tags = [i]) for i,data in enumerate(f)]
#print(type(trainings))
print("Doc２vec文書ベクトル用モデルに学習させたツイート数",len(trainings))
# print(trainings[:20])

#文書ベクトル用ツイートテキストの学習
model = Doc2Vec(
    documents= trainings, 
    dm = 1, 
    vector_size=300, 
    window=10, 
    alpha = 0.05, 
    min_count=1, 
    sample = 0, 
    workers=4, 
    epochs = 50
)

#出力用ディレクトリ作成（存在しない場合のみ）
def Make_Dir():
    new_dir_path = 'model'
    try:
        os.makedirs(new_dir_path)
    except FileExistsError:
        pass

# モデルのセーブ
Make_Dir()
model.save("./model/doc2vec.model")

# モデルのロード(モデルが用意してあれば、ここからで良い)
m = Doc2Vec.load('./model/doc2vec.model')

Doc２vec文書ベクトル用モデルに学習させたツイート数 260


## ４）ツイート予定文章のネガポジ予測を返す
　・データセットから、入力しておいたツイート予定文書と似ている文書を探す<br>
・ネガポジスコア付きで、類似ツイート上位１０個を返す<br>
#### 結果：成功。入力文書と同じツイート文が類似度１位に。ネガポジもデータズレなく表示できた

In [8]:
#類似判定と類似している上位10件の文書を出力

top10 = m.docvecs.most_similar(len(trainings) - 1)

print("=========== 判定したいツイート ===========\n")
print(similar)

print()
print("======= 類似度上位１０（全{}ツイート中） =======".format(len(trainings)))
print()
for i in range(len(top10)):
    score = top10[i]
    index = int(score[0])
    similar_score = score[1]
    tweet = df_tweet["Full_text"]
    judge = df_tweet["Judge"]
    posi_score = df_tweet["Posi_score"]
    nega_score = df_tweet["Nega_score"]
    print("…………　類似ツイート{}位：類似度 {:.4g}　…………".format((i+1), similar_score))
    print()
    print(tweet[index])
    print()
    print("【極性】：", judge[index])
    print("posi_score：",posi_score[index], "／", "nega_score：", nega_score[index])

    print()



=========== 判定したいツイート ===========

イオンマスク禁止従業員の人嫌がるのわかるわ。インフルで出校停止中なんだけど薬効いて体元気だからイオン遊ばせに来た。みたいな事凄く多いんだよ。『店員が媒介にならないよう全店でマスク奨励してます。ご理解下さい』ってアナウンスされる方が余程良いのでイオンさん、マスク禁止撤回して

======= 類似度上位１０（全260ツイート中） =======

…………　類似ツイート1位：類似度 0.9956　…………

イオンマスク禁止従業員の人嫌がるのわかるわ。

インフルで出校停止中なんだけど薬効いて体元気だからイオン遊ばせに来た。
みたいな事凄く多いんだよ。

『店員が媒介にならないよう全店でマスク奨励してます。ご理解下さい』
ってアナウンスされる方が余程良いのでイオンさん、マスク禁止撤回して

【極性】： negative
posi_score： 219.0 ／ nega_score： 257.0

…………　類似ツイート2位：類似度 0.9705　…………

マスクせずに咳してる人見ると、
「感染者だ！撃ち殺せ！」
「ですが！アイツはまだ人間じゃないですか！」
「馬鹿野郎！ここで殺らなきゃ、俺達の大事な奴まで感染者にされる可能性だってあるんだ！迷うな！引き金を引け！」
「畜生！感染者め！」
みたいな気持ちになる。マスクしやがり下さい。

【極性】： negative
posi_score： 119.0 ／ nega_score： 213.0

…………　類似ツイート3位：類似度 0.9566　…………

ご飯が出てくるのをいい子にして待ってますみたいな顔してるけど、そもそもそこはあなたの席ではありません https://t.co/o67uS6sBAq

【極性】： positive
posi_score： 84.0 ／ nega_score： 24.0

…………　類似ツイート4位：類似度 0.9501　…………

コミックマーケットでは深夜来場は認めておりません。絶対に深夜来場をしないようにお願いします。すみやかにお帰り下さい。 #C97 

【極性】： negative
posi_score： 70.0 ／ nega_score： 99.0

…………　類似ツイート5位：類似度 0.9481　……

# その他試みたこと
断念、または精度が全く良くない。覚書として記録

## １）文章ベクトルを特徴量としたネガポジ予測モデル
　・文章ベクトルとフォロワー数を特徴量X、ネガポジスコアを目的変数yとしたデータを学習<br>
　・文章ベクトルはDoc２vecとTf-idfの２種を作成<br>
　・ツイート予定文書を入力してネガポジスコアを予測する<br>
　・試した予測モデル<br>
　・MultiOutputRegressor、SVRのrbf と　SVRの線形、lightgbm、ランダムフォレスト<br>
#### 結果：精度が低すぎて断念<br>

### Doc2vecで文章ベクトル取得

In [9]:
#Doc2vecでベクトル化
from natto import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer

df_buzz = pd.read_csv('./output/buzz_tweet.csv',
                      usecols = ["Full_text", "Posi_score", "Nega_score", "Followers"])
#.to_csv('./output/for_training.csv', mode = "a", index = False, header = None)
#pd.read_csv('./output/fire_buzz_tweet.csv', usecols = ["Full_text", "Judge", "Sentiment"]).to_csv('./output/for_training.csv', mode = "a", index = False, header = None)
print("ベクトル化するセンチメントスコア付きデータ数：", len(df_buzz))
display(df_buzz.head())

#doc2vecでベクトル化
for_training = df_buzz['Full_text']
#print(for_training)
vector_tweet = []
for i in for_training:
    i = m.infer_vector(i)
    vector_tweet.append(i)

df_vector = pd.DataFrame(data = vector_tweet)

# print("Doc2vecベクトル")
# display(df_vector.head())

ベクトル化するセンチメントスコア付きデータ数： 314


Followers                                          Full_text  Nega_score  \
0     6800.0               君、すごい食い方やな\n https://t.co/yRTGvd43wT        31.0   
1     1564.0  絶対断らないと評判の病児保育室、助成金下りず2億円の赤字を出し閉鎖\n\n全国に2886カ所...       148.0   
2     2217.0  全国の皆さんへ\nどうか皆様のお力を貸してください。\n\n１日も早く娘を助けたいです。\n...        50.0   
3     3756.0  o0( 歴史上、さんざん他国の料理を魔改造してきた我が国が「寿司ポリス」などどは片腹痛い！あ...        85.0   
4     7363.0                   百合関係図です。 https://t.co/hZxIYOsSag        58.0   

   Posi_score  
0        75.0  
1       168.0  
2       129.0  
3       137.0  
4        82.0

### Tf-idfでベクトル取得
#### 参考サイト

機械学習_サポートベクターマシーン_pythonで実装<br>
https://dev.classmethod.jp/machine-learning/2017ad_20171214_svm_python/<br>
Tf-idfベクトルってなんだ？<br> https://qiita.com/MasatoTsutsumi/items/5b0a140b1ecbdd0396e1

In [10]:
# 2-1.tf-idf計算
from sklearn.feature_extraction.text import TfidfVectorizer

def Stop_Words():
    # ストップワードをダウンロード
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    urllib.request.urlretrieve(url, './output/stop_word.txt')

    with open('./output/stop_word.txt', 'r', encoding='utf-8') as file:
        stopwords = [word.replace('\n', '') for word in file.readlines()]

    #追加ストップワードを設定（助詞や意味のない平仮名１文字）
    add_words = ['あ','い','う','え','お','か','き','く','け','こ','さ','し','す','せ','そ','た','ち','つ','て','と',
                 'な','に','ぬ','ね','の','は','ひ','ふ','へ','ほ','ま','み','む','め','も','や','ゆ','よ',
                 'ら','り','る','れ','ろ','わ','を','ん','が','ぎ','ぐ','げ','ご','ざ','じ','ず','ぜ','ぞ',
                 'だ','ぢ','づ','で','ど','ば','び','ぶ','べ','ぼ','ぱ','ぴ','ぷ','ぺ','ぽ',
                 'くん','です','ます','ました','そして','でも','だから','だが','くらい','その','それ','かも',
                 'あれ','あの','あっ','そんな','この','これ','とか','とも','する','という','ござい',
                 'ので','なんて','たら', 'られ','たい','さて','てる','ください','なる','けど','でし',
                 'じゃん','だっ','なっ','でしょ', 'ある','って','こんな','ねえ'
                ]
    stopwords = stopwords + add_words
    return stopwords

stopwords = Stop_Words()
tfidfv = TfidfVectorizer(lowercase=True, stop_words=stopwords) # stop words処理
 
tfv_vector_fit = tfidfv.fit(for_training)
tfv_vector = tfv_vector_fit.transform(for_training)
print(tfv_vector.shape) 

# 2-2.次元削減(「lsa」を使って次元削減を行う)
from sklearn.decomposition import TruncatedSVD

# 2-2-1.パラメータの調整
list_n_comp = [5,10,50,100,500,1000] # 特徴量を何個に削減するか、というパラメータです。できるだけ情報量を欠損しないで、かつ次元数は少なくしたいですね。
for i in list_n_comp:
    lsa = TruncatedSVD(n_components=i,n_iter=5, random_state = 0)
    lsa.fit(tfv_vector) 
    tfv_vector_lsa = lsa.transform(tfv_vector)
    print('次元削減後の特徴量が{0}の時の説明できる分散の割合合計は{1}です'.format(i,round((sum(lsa.explained_variance_ratio_)),2)))

# 2-2-2.次元削減した状態のデータを作成
# 上記で確認した「n_components」に指定した上で、次元削減（特徴抽出）を行う
lsa = TruncatedSVD(n_components=1000, n_iter=5, random_state = 0) # 今回は次元数を1000に指定
lsa.fit(tfv_vector)
X_tf = lsa.transform(tfv_vector)
print()
print("次元削減後Tf-idf\n", X_tf.shape)
# print(X_tf)

(314, 2080)
次元削減後の特徴量が5の時の説明できる分散の割合合計は0.04です
次元削減後の特徴量が10の時の説明できる分散の割合合計は0.06です
次元削減後の特徴量が50の時の説明できる分散の割合合計は0.19です
次元削減後の特徴量が100の時の説明できる分散の割合合計は0.35です
次元削減後の特徴量が500の時の説明できる分散の割合合計は1.0です
次元削減後の特徴量が1000の時の説明できる分散の割合合計は1.0です

次元削減後Tf-idf
 (314, 314)


In [11]:
#X、yデータを作成

#Doc2vecのベクトルデータ
print("欠損値削除前データ", df_buzz.shape)
print()

#文書ベクトルを含んだdf
df_buzz_vec = pd.concat([df_buzz, df_vector], axis=1)
df_buzz_vec = df_buzz_vec.dropna(subset = ["Followers"])#欠損値行削除
df_buzz_vec = df_buzz_vec.drop([ "Full_text", "Nega_score", "Posi_score"], axis=1)
X = df_buzz_vec.values
print("Doc2vecベクトル")
print("X.shape", X.shape)
display(df_buzz_vec.head())

#tf-idfのベクトルデータ
tf_df = pd.DataFrame(data = X_tf)
tf_df = pd.concat([df_buzz, tf_df], axis=1)
tf_df = tf_df.dropna(subset = ["Followers"])#欠損値行削除
tf_df = tf_df.drop([ "Full_text", "Nega_score", "Posi_score"], axis=1)
X_tf_idf = tf_df.values
print("Tf-idfベクトル")
print("X_tf_idf.shape", tf_df.shape)
display(tf_df.head())

#yデータ作成
df_buzz = df_buzz.dropna(subset = ["Followers"])#y用に"Followers"の欠損行削除
y = df_buzz.loc[:,['Posi_score', 'Nega_score']]#できればDateも特徴量に入れたい
y_p = df_buzz['Posi_score']
y_n = df_buzz['Nega_score']
y = y.values
print()
print("y.shape", y.shape)
y_p = y_p.values
y_n = y_n.values

欠損値削除前データ (314, 4)

Doc2vecベクトル
X.shape (313, 301)


Followers         0         1         2         3         4         5  \
0     6800.0  0.044237  0.006614 -0.004481  0.002282  0.006894 -0.020148   
1     1564.0  0.021653  0.013749 -0.009543  0.000559 -0.005357 -0.013252   
2     2217.0  0.017007 -0.009793  0.001088 -0.000226  0.007482 -0.005318   
3     3756.0  0.034133  0.007313  0.003982 -0.020453  0.009534  0.005748   
4     7363.0  0.037321  0.001495 -0.004327  0.002138  0.000906 -0.011221   

          6         7         8    ...          290       291       292  \
0  0.007267  0.002130 -0.004062    ...    -0.004183  0.001240 -0.002830   
1  0.006427 -0.009228 -0.011012    ...    -0.014017  0.011406 -0.008416   
2  0.000894  0.015223  0.004249    ...     0.005082 -0.005141  0.001650   
3  0.013575 -0.001702 -0.012427    ...     0.006745  0.000688 -0.012102   
4  0.003346  0.004739 -0.003090    ...    -0.000518 -0.002960 -0.002879   

        293       294       295       296       297       298       299  
0 -0.013471 -0.004763 -0.005556 -0.000695  0.018570  0.001780  0.013421  
1 -0.024405  0.006039  0.017866 -0.006516  0.029692  0.010832 -0.000753  
2  0.004736  0.002359 -0.011970  0.007723  0.003055 -0.001900  0.005666  
3 -0.006678  0.006020  0.004849 -0.003342  0.043429  0.013293 -0.020528  
4 -0.005011 -0.000959 -0.006872  0.000163  0.008270  0.002082  0.010751  

[5 rows x 301 columns]

Tf-idfベクトル
X_tf_idf.shape (313, 315)


Followers         0         1         2         3             4         5  \
0     6800.0  0.291749 -0.000446 -0.000145 -0.002688 -5.854492e-17 -0.005929   
1     1564.0  0.174048 -0.000272 -0.000091 -0.001692  2.058179e-16 -0.003940   
2     2217.0  0.108983 -0.000176 -0.000061 -0.001153  4.200301e-16 -0.002961   
3     3756.0  0.111211 -0.000175 -0.000059 -0.001101 -1.461114e-16 -0.002614   
4     7363.0  0.291749 -0.000446 -0.000145 -0.002688 -1.506355e-16 -0.005929   

          6         7         8    ...              304       305       306  \
0 -0.011303 -0.006528 -0.012403    ...     1.418343e-16 -0.000029 -0.000063   
1 -0.007557 -0.004394 -0.008351    ...    -2.474998e-16 -0.000015 -0.000032   
2 -0.005744 -0.003388 -0.006440    ...    -1.162642e-17 -0.000007 -0.000017   
3 -0.005025 -0.002930 -0.005568    ...    -2.293239e-16 -0.000009 -0.000020   
4 -0.011303 -0.006528 -0.012403    ...     3.806199e-17 -0.000029 -0.000063   

        307       308       309       310       311           312       313  
0  0.000957 -0.003893  0.000312 -0.000123  0.000033 -3.550105e-17 -0.000026  
1  0.000496 -0.002028  0.000165 -0.000066  0.000018 -1.640593e-17 -0.000014  
2  0.000261 -0.001072  0.000089 -0.000036  0.000010  3.606252e-17 -0.000008  
3  0.000304 -0.001245  0.000102 -0.000041  0.000011  4.578045e-17 -0.000009  
4  0.000957 -0.003893  0.000312 -0.000123  0.000033  1.248560e-16 -0.000026  

[5 rows x 315 columns]


y.shape (313, 2)


### MultiOutputRegressorで複数の回帰¶
#### 結果：D2vベクトルよりTf-idfがややマシ

In [12]:
#MultiOutputRegressorで複数の回帰

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

#D2vベクトル
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=0)

#X, y = make_regression(n_samples=10, n_targets=3, random_state=1)
MOR = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X_train, y_train)
y_pred = MOR.predict(X_test)
score = MOR.score(X_test, y_test)

print("正解\n", y_test)
print()
print(y_pred)
print("R ^ 2_score(1に近いほど良い）：", score)
print()

#X_tf tf-idfベクトルを使った予測
X_train, X_test, y_train, y_test = train_test_split(
    X_tf_idf, y, train_size=0.7, test_size=0.3, random_state=0)

MOR = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X_train, y_train)
y_pred = MOR.predict(X_test)
score = MOR.score(X_test, y_test)
print(y_pred)
print("R ^ 2_score(1に近いほど良い）：", score)

正解
 [[  18.   46.]
 [  93.  125.]
 [  38.   41.]
 [ 110.   15.]
 [ 190.  157.]
 [ 140.  120.]
 [ 283.  219.]
 [  39.   48.]
 [ 114.   43.]
 [  21.    3.]
 [  59.   54.]
 [  13.    1.]
 [ 102.   81.]
 [ 180.   46.]
 [  73.  119.]
 [  71.   58.]
 [  35.   23.]
 [ 127.   89.]
 [  44.   30.]
 [  37.   49.]
 [  55.   61.]
 [ 122.   38.]
 [ 146.   61.]
 [  97.   78.]
 [  48.   37.]
 [ 160.  144.]
 [  94.   22.]
 [ 149.   50.]
 [  52.   15.]
 [  48.   39.]
 [ 247.   91.]
 [ 171.  121.]
 [ 125.   36.]
 [  68.   42.]
 [  73.   59.]
 [  94.   62.]
 [ 168.  197.]
 [  64.  143.]
 [  22.    4.]
 [  89.   95.]
 [  65.   79.]
 [ 205.   45.]
 [  50.   14.]
 [  44.   31.]
 [  19.   12.]
 [ 116.  142.]
 [  50.   19.]
 [  77.    5.]
 [ 126.  124.]
 [ 210.  169.]
 [ 103.   46.]
 [ 156.   20.]
 [  25.   27.]
 [  65.   43.]
 [ 118.  227.]
 [ 171.  214.]
 [  56.   23.]
 [ 114.  173.]
 [ 123.  139.]
 [  61.   46.]
 [ 117.   26.]
 [ 100.   79.]
 [  72.   42.]
 [  38.   42.]
 [ 166.  107.]
 [  88.   24.]
 [  85

### SVRのrbf と　SVRの線形で予測
#### 結果：予測値が全くダメ

In [13]:
#ポジ、ネガ別々で予測する
#SVRのrbf と　SVRの線形

import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt

#D2vベクトル(ポジのみ)
X_train, X_test, y_p_train, y_p_test = train_test_split(
    X, y_p, train_size=0.7, test_size=0.3, random_state=0)

svr_rbf = SVR(kernel='rbf', C=1, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1)
y_rbf = svr_rbf.fit(X_train, y_p_train)
y_lin = svr_lin.fit(X_train, y_p_train)

pred_rbf = svr_rbf.predict(X_test)
pred_lin = svr_lin.predict(X_test)

#精度

# 相関係数計算
rbf_corr = np.corrcoef(y_p_test, pred_rbf)[0, 1]
lin_corr = np.corrcoef(y_p_test, pred_lin)[0, 1]

# RMSEを計算（０に近いほど良い）
rbf_rmse = sqrt(mean_squared_error(pred_rbf, y_p_test))
lin_rmse = sqrt(mean_squared_error(pred_lin, y_p_test))

print("RBF: RMSE（０に近いほど良い） {} ".format(rbf_rmse))
print("Linear: RMSE（０に近いほど良い） {}" .format(lin_rmse))
print()
print("正解", y_p_test)
print("rbf推定", pred_rbf)
print("lin推定", pred_lin)


RBF: RMSE（０に近いほど良い） 63.28208955684835 
Linear: RMSE（０に近いほど良い） 635.1003506204574

正解 [  18.   93.   38.  110.  190.  140.  283.   39.  114.   21.   59.   13.
  102.  180.   73.   71.   35.  127.   44.   37.   55.  122.  146.   97.
   48.  160.   94.  149.   52.   48.  247.  171.  125.   68.   73.   94.
  168.   64.   22.   89.   65.  205.   50.   44.   19.  116.   50.   77.
  126.  210.  103.  156.   25.   65.  118.  171.   56.  114.  123.   61.
  117.  100.   72.   38.  166.   88.   85.  119.  141.  186.   81.   79.
   87.  266.   95.   53.  214.   76.  120.  147.   70.  122.   45.   44.
  162.  128.  116.  116.  197.   92.  345.   63.   90.   84.]
rbf推定 [ 91.18305705  91.58561873  91.58561873  91.58561873  90.68690313
  91.58561873  91.58561873  91.58561873  91.58561873  91.58561873
  91.58561872  91.58561873  91.58561873  91.99073469  91.66711441
  91.58561873  91.58561873  93.12698319  91.61286514  91.58561873
  91.58557377  91.58561873  91.58561873  91.58561873  91.58561873
  91.58

### lightgbm
#### 参考サイト

Mercari Price Challenge -機械学習を使ったメルカリの価格予測 Ridge回帰 LightGBM

http://rautaku.hatenablog.com/entry/2017/12/22/195649

#### 結果：RMSEが0には程遠い

In [14]:
#必要なツールをインストール(初回のみ実行)
! pip install lightgbm

In [15]:
#LightGBM を使った回帰予測(D2Vベクトル)

import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

def main():
    #D2vベクトル(ポジのみ)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_p, train_size=0.7, test_size=0.3, random_state=0)

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # LightGBM のハイパーパラメータ
    lgbm_params = {
        # 回帰問題
        'objective': 'regression',
        # RMSE (平均二乗誤差平方根) の最小化を目指す
        'metric': 'rmse',
    }

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train, 
                      valid_sets=lgb_eval, num_boost_round=8000, 
                      early_stopping_rounds=5000, verbose_eval=500)

    # テストデータを予測する
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # RMSE を計算する
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("RMSE（０に近いほど良い）", rmse)

if __name__ == '__main__':
    main()

Training until validation scores don't improve for 5000 rounds
[500]	valid_0's rmse: 62.2081
[1000]	valid_0's rmse: 62.2405
[1500]	valid_0's rmse: 62.2444
[2000]	valid_0's rmse: 62.2453
[2500]	valid_0's rmse: 62.2454
[3000]	valid_0's rmse: 62.2454
[3500]	valid_0's rmse: 62.2454
[4000]	valid_0's rmse: 62.2454
[4500]	valid_0's rmse: 62.2454
[5000]	valid_0's rmse: 62.2454
Early stopping, best iteration is:
[19]	valid_0's rmse: 59.5933
RMSE（０に近いほど良い） 59.5933203655


In [16]:
#LightGBM を使った回帰予測（Tfーidfベクトル）

def main():

    #X_tf tf-idfベクトルを使った予測(ポジのみ)
    X_train, X_test, y_train, y_test = train_test_split(
        X_tf_idf, y_p, train_size=0.7, test_size=0.3, random_state=0)

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # LightGBM のハイパーパラメータ
    lgbm_params = {
        # 回帰問題
        'objective': 'regression',
        # RMSE (平均二乗誤差平方根) の最小化を目指す
        'metric': 'rmse',
    }
    
    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train, 
                      valid_sets=lgb_eval, num_boost_round=8000, 
                      early_stopping_rounds=5000, verbose_eval=500)
#     model = lgb.LGBMRegressor()
#     model.fit(X_train, y_train)

    # テストデータを予測する
    y_pred = model.predict(X_test)

    # RMSE を計算する
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("RMSE（０に近いほど良い）",rmse)


if __name__ == '__main__':
    main()

Training until validation scores don't improve for 5000 rounds
[500]	valid_0's rmse: 67.0909
[1000]	valid_0's rmse: 67.1183
[1500]	valid_0's rmse: 67.1202
[2000]	valid_0's rmse: 67.1205
[2500]	valid_0's rmse: 67.1206
[3000]	valid_0's rmse: 67.1206
[3500]	valid_0's rmse: 67.1206
[4000]	valid_0's rmse: 67.1206
[4500]	valid_0's rmse: 67.1206
[5000]	valid_0's rmse: 67.1206
Early stopping, best iteration is:
[10]	valid_0's rmse: 60.8126
RMSE（０に近いほど良い） 60.8125701284


### ランダムフォレスト

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

#D2vベクトル
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=0)
# ランダムフォレスト回帰オブジェクト生成
rfr = RandomForestRegressor(n_estimators=100)
# 学習の実行
rfr.fit(X_train, y_train)
# テストデータで予測実行
predict_y = rfr.predict(X_test)
# R2決定係数で評価
r2_score = r2_score(y_test, predict_y)
print("R^2(1に近いほど良い）:", r2_score)


R^2(1に近いほど良い）: -0.0938475289851


## ２）ツイッターAPI制限への挑戦（データセットの拡大）
　・古いツイートを大量取得できるパッケージを発見（通常は１週間程度しか遡れない）<br>
#### 結果：取得データから反応ツイートの取得を試みたができなかった<br>

### GetOldTweets3 0.0.11
古いツイートをトークン申請なしで大量取得できるパッケージ<br>
https://pypi.org/project/GetOldTweets3/

In [18]:
#必要なツールをインストール(初回のみ実行)
! pip install GetOldTweets3

In [19]:
#指定日のトップツイートを取得、'./output/toptweets.csv'に保存
! GetOldTweets3 --lang ja  --toptweets  --querysearch "" --since 2019-2-10 --until 2019-2-11 --output './output/toptweets.csv'


Saved 1800^C

Interrupted.


Done. Output file generated "./output/toptweets.csv".
